In [10]:
import os
import glob
import numpy as np
import pandas as pd
from functions import split_preds, metrics, measure_csv_df
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
os.chdir("/Users/pestephan/Documents/GitHub/Machine-Learning-using-Sports-Data/Predicting-PremLeague-Outcomes-With-ML/files/Datasets")

Data Taken from http://www.football-data.co.uk/englandm.php

In [2]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
#combine all files in the list
master_sheet = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
master_sheet.to_csv( "master_sheet.csv", index=False, encoding='utf-8-sig')

Reduce table to using columns needed for the model

Columns Required:

Date, Time HomeTeam, AwayTeam, FullTimeHG, FullTimeAG, FullTimeRresults, HalfTimeHG, HalfTimeAG, HomeShots, AwayShots, HomeShotsTarger, AwayShotsTarger, HomeCorners, AwayCorners, HomeFouls, AwayFouls, HomeRed, AwayRed

Columns To Consider:

Attendance, Referee

In [4]:
subset_master = master_sheet[["Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HS", "AS", "HST", "AST", "HC", "AC", "HF", "AF", "HR", "AR"]]
subset_master = subset_master.fillna(0)
# 0 = Home Wins, 1 = Draw, 2 = Away Wins
subset_master.FTR.replace(['H', 'D', 'A'], [0, 1, 2], inplace = True)

In [5]:
subset_master.head()

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST,HC,AC,HF,AF,HR,AR
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,13.0,8.0,4.0,12.0,6.0,14.0,10.0,0.0,0.0
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,7.0,9.0,1.0,2.0,3.0,10.0,18.0,0.0,0.0
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,15.0,4.0,6.0,6.0,3.0,17.0,12.0,0.0,0.0
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,9.0,1.0,4.0,2.0,5.0,9.0,11.0,0.0,1.0
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,13.0,7.0,7.0,3.0,2.0,9.0,16.0,1.0,0.0


In [6]:
msk = np.random.rand(len(subset_master)) < 0.75

train_df = subset_master[msk]
test_df = subset_master[~msk]

X_train = train_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG"])
y_train = train_df[['FTR']] 
X_test = test_df.drop(columns = ['Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', "HTHG", "HTAG", ])
y_test = test_df[['FTR']]

In [11]:
clf = RandomForestClassifier(n_estimators = 2000, max_depth=3, random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict_proba(X_test)
y_pred = pd.DataFrame(y_pred)
split_preds(subset_master, y_pred)

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AST,HC,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,...,4.0,12.0,6.0,14.0,10.0,0.0,0.0,0.650492,0.217608,0.131900
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,...,1.0,2.0,3.0,10.0,18.0,0.0,0.0,0.281812,0.248569,0.469619
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,...,6.0,6.0,3.0,17.0,12.0,0.0,0.0,0.375643,0.239970,0.384387
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,...,4.0,2.0,5.0,9.0,11.0,0.0,1.0,0.442755,0.241175,0.316070
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,...,7.0,3.0,2.0,9.0,16.0,1.0,0.0,0.410651,0.239272,0.350077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,24/05/15,0,Leicester,QPR,5.0,1.0,0,2.0,0.0,22.0,...,2.0,5.0,6.0,7.0,6.0,0.0,0.0,0.285770,0.247556,0.466674
377,24/05/15,0,Man City,Southampton,2.0,0.0,0,1.0,0.0,15.0,...,4.0,8.0,4.0,13.0,8.0,0.0,0.0,0.702407,0.193107,0.104486
378,24/05/15,0,Newcastle,West Ham,2.0,0.0,0,0.0,0.0,17.0,...,1.0,2.0,3.0,9.0,9.0,0.0,0.0,0.327913,0.291836,0.380251
379,24/05/15,0,Stoke,Liverpool,6.0,1.0,0,5.0,0.0,15.0,...,4.0,3.0,9.0,13.0,4.0,0.0,0.0,0.427549,0.245554,0.326897


Identifying how often Predicted Results decided the outcome (win, draw, away) against the actual results)

In [13]:
metrics(subset_master)

False    55.822621
True     44.177379
Name: Comparison, dtype: float64
0    45.285564
2    30.164600
1    24.549836
Name: FTR, dtype: float64
0    92.679353
2     7.320647
Name: PredResult, dtype: float64


In [14]:
subset_master

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,...,AC,HF,AF,HR,AR,HomeWinPerc,DrawPerc,AwayPerc,PredResult,Comparison
0,13/08/11,0,Blackburn,Wolves,1.0,2.0,2,1.0,1.0,16.0,...,6.0,14.0,10.0,0.0,0.0,0.650492,0.217608,0.131900,0,False
1,13/08/11,0,Fulham,Aston Villa,0.0,0.0,1,0.0,0.0,13.0,...,3.0,10.0,18.0,0.0,0.0,0.281812,0.248569,0.469619,2,False
2,13/08/11,0,Liverpool,Sunderland,1.0,1.0,1,1.0,0.0,11.0,...,3.0,17.0,12.0,0.0,0.0,0.375643,0.239970,0.384387,2,False
3,13/08/11,0,Newcastle,Arsenal,0.0,0.0,1,0.0,0.0,6.0,...,5.0,9.0,11.0,0.0,1.0,0.442755,0.241175,0.316070,0,False
4,13/08/11,0,QPR,Bolton,0.0,4.0,2,0.0,1.0,13.0,...,2.0,9.0,16.0,1.0,0.0,0.410651,0.239272,0.350077,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,24/05/15,0,Leicester,QPR,5.0,1.0,0,2.0,0.0,22.0,...,6.0,7.0,6.0,0.0,0.0,0.285770,0.247556,0.466674,2,False
377,24/05/15,0,Man City,Southampton,2.0,0.0,0,1.0,0.0,15.0,...,4.0,13.0,8.0,0.0,0.0,0.702407,0.193107,0.104486,0,True
378,24/05/15,0,Newcastle,West Ham,2.0,0.0,0,0.0,0.0,17.0,...,3.0,9.0,9.0,0.0,0.0,0.327913,0.291836,0.380251,2,False
379,24/05/15,0,Stoke,Liverpool,6.0,1.0,0,5.0,0.0,15.0,...,9.0,13.0,4.0,0.0,0.0,0.427549,0.245554,0.326897,0,True


Columns Required for predicting:

HomeShots, AwayShots, HomeShotsTarger, AwayShotsTarger, HomeCorners, AwayCorners, HomeFouls, AwayFouls, HomeRed, AwayRed

In [15]:
current_season = pd.read_csv("/Users/pestephan/Documents/GitHub/Machine-Learning-using-Sports-Data/Predicting-PremLeague-Outcomes-With-ML/2020-2021.csv")

In [19]:
y_pred = clf.predict_proba(current_season)
y_pred = pd.DataFrame(y_pred)
split_preds(subset_master, y_pred)

ValueError: could not convert string to float: '12/09/2020'

In [18]:
current_season = measure_csv_df(current_season)
current_season.FTR.replace(['H', 'D', 'A'], [0, 1, 2], inplace = True)
current_season.head()

TypeError: measure_csv_df() missing 1 required positional argument: 'clf'